In [1]:
import json
import os
import requests
import zipfile

from datetime import datetime

In [11]:
os.getcwd()

'd:\\tmp\\_reveng\\audioeffect-qm\\resae\\recommendbase'

In [16]:
os.chdir('..')

In [2]:
src = 'resae/ugc.json'
d = json.load(open(src))

dst = os.path.splitext(src)[0]
os.makedirs(dst, exist_ok=True)
os.chdir(dst)

for ae in d['ugcAudioEffectItems']:

    ae_id = ae['effectId']
    ae_name = ae['effectName']
    ae_url = ae['effectDownloadLink']
    ae_ts = datetime.strptime(ae['createTime'], '%Y-%m-%d %H:%M:%S').timestamp()
    target = f'{ae_id:04d}-{ae_name}.zip'

    print('Downloading', target)
    response = requests.get(ae_url)
    with open(target, 'wb') as f:
        f.write(response.content)
    os.utime(target, (ae_ts, ae_ts))

    with zipfile.ZipFile(target, 'r') as fzip:
        if len(fzip.namelist()) != 2:
            continue
        print('Extracting', target)
        fzip.extractall()

    try:
        target_new = f'{ae_id:04d}-{ae_name}.qmaep'
        os.rename('AudioEffectPackage/AudioEffect.qmaep', target_new)
        os.utime(target_new, (ae_ts, ae_ts))
        os.remove(target)
    except:
        pass

# os.remove('AudioEffectPackage')
os.chdir('../..')

Extracting 7513-回音壁音效模拟.zip
Extracting 4055-醇厚暖性人声.zip
Extracting 0869-AirPodsPro增进.zip
Extracting 0335-现场演唱会清澈版.zip
Extracting 0046-演唱现场.zip
Extracting 6977-录音棚·空间还原.zip
Extracting 6230-大型礼堂现场混响.zip
Extracting 5978-醇低音-耳机空间音效.zip
Extracting 4848-灵动回响.zip
Extracting 1127-HP专业音效.zip
Extracting 3062-立体环绕高保真.zip
Extracting 0653-男声弱混JJ定制.zip
Extracting 4753-link极致享受.zip
Extracting 3055-超宽低音.zip
Extracting 2641-复古震撼低音.zip
Extracting 7869-IER-Z1R御用高保真.zip
Extracting 7226-Q弹近场.zip
Extracting 5953-专业差分环绕2.0.zip
Extracting 4758-超级HIFI.zip
Extracting 1703-清醒梦.zip
Extracting 2299-杰伦专享音效.zip
Extracting 0203-灵悦宽音.zip
Extracting 1758-HIFI全景环绕.zip
Extracting 0210-3D舒适立体声.zip
Extracting 1250-影院音响现场.zip
Extracting 4517-环绕超重低音.zip
Extracting 4396-HUAONE混音.zip
Extracting 0437-疯狂现场.zip
Extracting 0412-疯狂人声.zip
Extracting 8373-纯氧动态环绕人声.zip
Extracting 3295-悦享ACG.zip
Extracting 3007-完美环绕清澈人声.zip
Extracting 7859-完美清澈人声.zip
Extracting 6060-悠扬古风.zip
Extracting 2457-至尊重低音.zip
Extracting 2358-重低音强化.zip
Extracting

In [2]:
src = 'resae/soundbox.json'
d = json.load(open(src))
ts = d['timeStamp']

dst = os.path.splitext(src)[0]
os.makedirs(dst, exist_ok=True)
os.chdir(dst)

for v in d['vendorItems']:

    v_id = v['vendorId']
    v_name = v['vendorName']

    for m in v['modelItems']:

        m_id = m['modelId']
        m_name = m['modelName']
        m_url = m['modelAEPLink']
        target = f'{v_id:03d}-{v_name}_{m_id:04d}-{m_name}.aep'

        print('Downloading', target)
        response = requests.get(m_url)
        with open(target, 'wb') as f:
            f.write(response.content)
        os.utime(target, (ts, ts))

os.chdir('../..')

In [2]:
import pandas as pd

src = 'resae/headphone.json'
d = json.load(open(src))
ts = d['timeStamp']

dst = os.path.splitext(src)[0] + '.csv'

data = []
for v in d['vendorItems']:
    for m in v['modelItems']:
        data.append({
            'vendorId': v['vendorId'],
            'vendorName': v['vendorName'],
            'modelId': m['modelId'],
            'modelName': m['modelName'],
            'modelEQ': m['modelEQ']
        })

df = pd.DataFrame(data)
df.sort_values(by=['vendorId', 'modelId']).to_csv(dst, index=False)
os.utime(dst, (ts, ts))

In [2]:
src = 'resae/recommendbase.json'
d = json.load(open(src))
ts = d['timeStamp']

dst = os.path.splitext(src)[0]
os.makedirs(dst, exist_ok=True)
os.chdir(dst)

for ae in d['effectBaseItems']:

    ae_id = ae['effectId']
    ae_name = ae['effectName']
    irs = ae['effectIRLinks']
    if not irs: continue

    for i, ir in enumerate(irs):

        ir_url = ir['irLink']
        ext = os.path.splitext(ir['irFileName'])[1]
        number = f'-{(i+1)}' if len(irs) > 1 else ''
        target = f'{ae_id:03d}-{ae_name}{number}{ext}'

        print('Downloading', target)
        response = requests.get(ir_url)
        with open(target, 'wb') as f:
            f.write(response.content)
        os.utime(target, (ts, ts))

os.chdir('../..')

In [3]:
src = 'resae/DJRemix.json'
d = json.load(open(src))

dst = os.path.splitext(src)[0]
os.makedirs(dst, exist_ok=True)
os.chdir(dst)

for tmp in d['template']:

    tmp_name = tmp['name']
    tmp_url = tmp['link']
    target = f'{tmp_name}.zip'

    print('Downloading', target)
    response = requests.get(tmp_url)
    with open(target, 'wb') as f:
        f.write(response.content)

os.chdir('../..')